# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * FastText
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

from numba import jit
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import FastText, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\pc\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [4]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Load Data

In [5]:
#Naver
naverData = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
naverData = pd.DataFrame.from_dict(naverData, orient = 'index')
naverData.reset_index(inplace = True)
naverData.rename(columns = {'index' : 'id'}, inplace = True)
#Daum
daumData = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
daumData = pd.DataFrame.from_dict(daumData, orient = 'index')
daumData.reset_index(inplace = True)
daumData.rename(columns = {'index' : 'id'}, inplace = True)

print ('Naver : {}'.format(naverData.shape))
print ('Daum : {}'.format(daumData.shape))

Naver : (15120, 11)
Daum : (9372, 11)


In [6]:
extNaverData = naverData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extNaverData['site'] = pd.Series(['Naver']*extNaverData.shape[0])
extDaumData = daumData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extDaumData['site'] = pd.Series(['daum']*extDaumData.shape[0])

In [7]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    #newsPath = '/Volumes/data/pre_data/news_sentiment/'
    newsPath = './data/pre_data/news_sentiment/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'

## TaggedDocument For FastText

In [8]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## FastText Model

### Twitter

In [9]:
taggerName = 'ct'

#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [10]:
trainName = './data/pre_data/train_test_Data/pre_data_train_for_fastText_sentiment_by_ct.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

100%|██████████| 442359/442359 [00:00<00:00, 1334417.99it/s]


(442359, 159010)
vocab size : 159010


#### News to tagged Document

In [11]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_fastText_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, ct, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_fastText_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_fastText_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, ct, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_fastText_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [12]:
print ('Load Model1')
model1 = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
print ('Load Model2')
model2 = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
print ('Load Model3')
model3 = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

Load Model1
Load Model2
Load Model3


#### model1

In [13]:
modelName = bm.Return_ModelName('fastText', model1,'ct')

In [14]:
classifierList = glob(classifierPath+'*'+modelName)

In [15]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
Instructions for updating:
keep_dims is deprecated, use keepdims instead
NeuralNetwork_1
Instructions for updating:
keep_dims is deprecated, use keepdims instead
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [16]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:07<00:00, 20787.15it/s]


running time : 0:00:07.914689
Vectorizing Data


9372it [16:48,  9.29it/s]


scaling Data
total running time : 0:16:56.839356


In [17]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 623853.63it/s]


9372/9372 [==============================] - ETA: 40 - ETA: 0 - ETA:  - ETA:  - 0s 33us/step


9372it [00:00, 44971.77it/s]


9372/9372 [==============================] - ETA: 1: - ETA: 1s - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 57us/step


9372it [00:00, 176563.40it/s]


Wall time: 21.5 s


In [18]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:02<00:00, 70060.78it/s]


running time : 0:00:02.325316
Vectorizing Data


15120it [11:31, 21.85it/s]


scaling Data
total running time : 0:11:35.294469


In [19]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 350855.74it/s]


15120/15120 [==============================] - ETA: 2: - ETA: 3s - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 64us/step


15120it [00:00, 629008.32it/s]


15120/15120 [==============================] - ETA: 49 - ETA: 1 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 31us/step


15120it [00:00, 629058.23it/s]


Wall time: 30.9 s


In [20]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [21]:
modelName = bm.Return_ModelName('fastText', model2,'ct')

In [22]:
classifierList = glob(classifierPath+'*'+modelName)

In [23]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [24]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:05<00:00, 27980.10it/s]


running time : 0:00:05.853044
Vectorizing Data


9372it [05:33, 28.13it/s]


scaling Data
total running time : 0:05:39.259452


In [25]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 623823.93it/s]


9372/9372 [==============================] - ETA: 32 - ETA: 0 - ETA:  - ETA:  - 0s 29us/step


9372it [00:00, 222804.87it/s]


9372/9372 [==============================] - ETA: 16 - ETA: 0 - ETA:  - ETA:  - ETA:  - 0s 36us/step


9372it [00:00, 584877.28it/s]


Wall time: 20.6 s


In [26]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:01<00:00, 105873.29it/s]


running time : 0:00:01.539473
Vectorizing Data


15120it [03:05, 81.54it/s]


scaling Data
total running time : 0:03:07.277000


In [27]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 308099.05it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 19us/step


15120it [00:00, 603847.50it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 24us/step


15120it [00:00, 628790.03it/s]


Wall time: 32.6 s


In [28]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [29]:
modelName = bm.Return_ModelName('fastText', model3,'ct')

In [30]:
classifierList = glob(classifierPath+'*'+modelName)

In [31]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [32]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:02<00:00, 57702.31it/s]


running time : 0:00:02.822298
Vectorizing Data


9372it [04:00, 39.04it/s]


scaling Data
total running time : 0:04:03.101711


In [33]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 359929.84it/s]


9372/9372 [==============================] - ETA: 26 - ETA: 0 - ETA:  - ETA:  - 0s 27us/step


9372it [00:00, 623883.33it/s]


9372/9372 [==============================] - ETA: 25 - ETA: 0 - ETA:  - ETA:  - ETA:  - 0s 33us/step


9372it [00:00, 623853.63it/s]


Wall time: 23.4 s


In [34]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

FastText(vocab=162564, size=1000, alpha=0.025)


100%|██████████| 162564/162564 [00:01<00:00, 114794.48it/s]


running time : 0:00:01.421138
Vectorizing Data


15120it [03:25, 73.73it/s]


scaling Data
total running time : 0:03:26.819750


In [35]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 629076.95it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 18us/step


15120it [00:00, 628796.27it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 25us/step


15120it [00:00, 628790.03it/s]


Wall time: 37.3 s


In [36]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

### Mecab

In [9]:
taggerName = 'mecab'

#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [10]:
trainName = './data/pre_data/train_test_Data/pre_data_train_for_fastText_sentiment_by_mecab.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

100%|██████████| 442359/442359 [00:00<00:00, 1482191.56it/s]


(442359, 162003)
vocab size : 162003


#### News to tagged Document

In [11]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_fastText_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, mecab, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_fastText_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, mecab, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_fastText_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [12]:
print ('Load Model1')
model1 = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-mecab.model')
print ('Load Model2')
model2 = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-mecab.model')
print ('Load Model3')
model3 = FastText.load(loadModelPath+'fastText_size-1000_epoch-20_ngrams-3_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-mecab.model')

Load Model1
Load Model2
Load Model3


#### model1

In [13]:
modelName = bm.Return_ModelName('fastText', model1,'mecab')

In [14]:
classifierList = glob(classifierPath+'*'+modelName)

In [15]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
Instructions for updating:
keep_dims is deprecated, use keepdims instead
NeuralNetwork_1
Instructions for updating:
keep_dims is deprecated, use keepdims instead
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [16]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:05<00:00, 30233.65it/s]


running time : 0:00:05.661371
Vectorizing Data


9372it [17:47,  8.78it/s]


scaling Data
total running time : 0:17:53.867761


In [17]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 343331.18it/s]


9372/9372 [==============================] - ETA: 37 - ETA: 0 - ETA:  - ETA:  - ETA:  - 0s 36us/step


9372it [00:00, 143966.40it/s]


9372/9372 [==============================] - ETA: 41 - ETA: 3 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 54us/step


9372it [00:00, 119787.23it/s]


Wall time: 24.2 s


In [18]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:03<00:00, 51421.00it/s]


running time : 0:00:03.228807
Vectorizing Data


15120it [07:09, 35.20it/s]


scaling Data
total running time : 0:07:13.156170


In [19]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 629070.71it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 17us/step


15120it [00:00, 603657.83it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 35us/step


15120it [00:00, 126679.43it/s]


Wall time: 27 s


In [20]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [21]:
modelName = bm.Return_ModelName('fastText', model2,'mecab')

In [22]:
classifierList = glob(classifierPath+'*'+modelName)

In [23]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [24]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:03<00:00, 46049.51it/s]


running time : 0:00:03.605978
Vectorizing Data


9372it [06:48, 22.97it/s]


scaling Data
total running time : 0:06:51.856900


In [25]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 641078.61it/s]


9372/9372 [==============================] - ETA: 22 - ETA: 0 - ETA:  - ETA:  - 0s 26us/step


9372it [00:00, 623893.23it/s]


9372/9372 [==============================] - ETA: 15 - ETA: 0 - ETA:  - ETA:  - ETA:  - 0s 29us/step


9372it [00:00, 668486.59it/s]


Wall time: 20.3 s


In [26]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:01<00:00, 107998.55it/s]


running time : 0:00:01.539425
Vectorizing Data


15120it [05:40, 44.40it/s]


scaling Data
total running time : 0:05:42.461675


In [27]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 656425.01it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 17us/step


15120it [00:00, 656418.21it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 26us/step


15120it [00:00, 629083.19it/s]


Wall time: 32.5 s


In [28]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [29]:
modelName = bm.Return_ModelName('fastText', model3,'mecab')

In [30]:
classifierList = glob(classifierPath+'*'+modelName)

In [31]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression
NeuralNetwork_1
NeuralNetwork_2
RandomForestClassifier
SVC
XGBoost


In [32]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:04<00:00, 39179.53it/s]


running time : 0:00:04.236385
Vectorizing Data


9372it [04:50, 32.29it/s]


scaling Data
total running time : 0:04:54.642550


In [33]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 584885.98it/s]


9372/9372 [==============================] - ETA: 22 - ETA: 0 - ETA:  - ETA:  - 0s 25us/step


9372it [00:00, 668520.70it/s]


9372/9372 [==============================] - ETA: 23 - ETA: 0 - ETA:  - ETA:  - ETA:  - 0s 33us/step


9372it [00:00, 623883.33it/s]


Wall time: 22.1 s


In [34]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

FastText(vocab=165823, size=1000, alpha=0.025)


100%|██████████| 165823/165823 [00:01<00:00, 121210.36it/s]


running time : 0:00:01.372066
Vectorizing Data


15120it [04:15, 59.10it/s]


scaling Data
total running time : 0:04:17.519591


In [35]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extNaverData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 629045.75it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 17us/step


15120it [00:00, 603657.83it/s]


15120/15120 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 25us/step


15120it [00:00, 503232.61it/s]


Wall time: 35.5 s


In [36]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict